In [1]:
from utils import get_spark_session
import time
from data_generator import gen_bulk_data

spark = get_spark_session("iceberg_table_partitioning")
df_test = gen_bulk_data(spark, 10**4)

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-21b63894-4803-45fd-9c1a-b16d0dac49bc;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.95.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found softwa

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: double (nullable = true)
 |-- last_purchase: timestamp (nullable = true)
 |-- last_purchase_date: string (nullable = true)



# Time Travel

- **What is it?** The ability to query the table as it was during a previous valid snapshot;
- **Why use it?**:
    - To run ML model tests on the same data it was tested previously;
    - Avoid the need for snapshotting the table with CTAS statements;
    - For quality control to assess data before and after updates;
      
- **Limitations**: Can only time-travel to valid snapshots (not expired ones);
- 



In [ ]:
spark.read.option("as-of-timestamp", 499162860000).format("iceberg").load("path/to/table")
spark.read.option("snapshot-id", 10963874102873L).format("iceberg").load("path/to/table")

# Iceberg Tables Maintainance

## 1. Expiring Snapshots

```sql
CALL prod.system.expire_snapshots(
    table => 'db.sample',
    older_than => now(),
    retain_last => 30
);
CALL prod.system.expire_snapshots('db.sample', TIMESTAMP '2024-09-01 00:00:00.000', 100);
```

## 2. Rewriting Data Files and Manifests (Compaction)

- To avoid the small file problem, clean up deleted records or optimize the data further with sorting we can re-write data files;
- In the example below we rewrite the data files using a "sort" strategy which sorts the records along with rewriting item.
- The above strategy will take longer than the "binpack" strategy which mainly rewrites data without sorting.

### Rewriting data files and Manifests

```sql
CALL catalog.system.rewrite_data_files(
    table => 'db.sample',
    strategy => 'sort',
    sort_order => 'id DESC NULL LAST, name ASC NULLS FIRST'
```
### Rewriting Manifests only

```sql
CALL catalog.system.rewrite_manifests('db_sample')
```

## 3. Deleting Orphan Files

- Running **remove_orphan_files** procedure will allow you to clean up any files that may have not been deleted while expiring snapshots;
- These files not associated with your table are called orphan files;
- By default will look in the tables data directory, but in case there are data files in other directories (because of migrate or add_files) you can also specify a directory to clean;


```sql
CALL catalog.system.remove_orphan_files(table => 'db.sample')
```


In [1]:
spark.stop()

NameError: name 'spark' is not defined